<p>

</p>

## **LlamaIndex-RAG**
Perform RAG (Retrieval-Augmented Generation) from your PDFs using this Colab notebook!
<br><br>

## **Features**
- Free, no API or Token required
- Fast inference on Colab's free T4 GPU
- Powered by Hugging Face quantized LLMs (llama-cpp-python)
- Powered by Hugging Face local text embedding models
- Set custom prompt templates
- Prepared Chat mode (not QA)
<br><br>

## **Getting started**
1. Make sure the Colab's Runtime Type is set to T4 GPU (at least)
2. Edit preferences in Block 4
3. Upload your PDF into given directory form

<br><br>



### Mount drive

In [1]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip -q install llama-index pypdf
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -q install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.0/630.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [3]:
# because updates there was need to use legacy part of libary
import os
import time
from glob import glob

# from llama_index.core import Prompt, StorageContext, load_index_from_storage, ServiceContext, VectorStoreIndex, SimpleDirectoryReader, set_global_tokenizer
# from llama_index.core.prompts import PromptTemplate

from llama_index.legacy import Prompt, StorageContext, load_index_from_storage, ServiceContext, VectorStoreIndex, SimpleDirectoryReader, set_global_tokenizer
from llama_index.legacy.prompts import PromptTemplate

from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.legacy.llms import LangChainLLM, HuggingFaceLLM, LlamaCPP, ChatMessage, MessageRole
from llama_index.legacy.chat_engine.condense_question import CondenseQuestionChatEngine

from transformers import AutoTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Preference settings - change as desired

# pdf_path = '/content/rag_data.pdf' original
# pdf_path = '/content/drive/MyDrive/RAG/raq_data.pdf'
text_embedding_model = 'thenlper/gte-base'  #Alt: thenlper/gte-base, jinaai/jina-embeddings-v2-base-en
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'
# set_global_tokenizer(AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode)

In [11]:
#@title Choose directory with pdfs to RAG { display-mode: "form" }

pdf_directory = "/content/drive/MyDrive/RAG/pdf" #@param [""] {allow-input: true}

# Use glob to get a list of all PDF files in the specified directory
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Define the lambda function to extract file metadata
filename_fn = lambda pdf_path: {'file_name': os.path.basename(pdf_path)}

# Create the SimpleDirectoryReader with the list of PDF files and the metadata function
loader = SimpleDirectoryReader(input_files=pdf_files, file_metadata=filename_fn)

# Load the data
documents = loader.load_data()

In [6]:
# Load models and service context
embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
llm = LlamaCPP(model_url=llm_url, temperature=0.7, max_new_tokens=256, context_window=4096, generate_kwargs = {"stop": ["<s>", "[INST]", "[/INST]"]}, model_kwargs={"n_gpu_layers": -1}, verbose=True)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

total size (MB): 4081.0


3892it [00:33, 117.34it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loa

In [12]:
# Indexing
start_time = time.time()

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed indexing time: {elapsed_time:.2f} s")

Elapsed indexing time: 62.02 s


## choose one

### Prompt Template (RAG)

In [8]:
# Prompt Template (RAG)
text_qa_template = Prompt("""
<s>[INST] <<SYS>>
You are the  summarize the findings of the review  in max 10 points short sentences
<</SYS>>



Continue the conversation: {query_str}
""")
# text_qa_template = Prompt("""
# <s>[INST] <<SYS>>
# You are an assistant chatbot assigned to a doctor and your objective is to collect information from the patient for the doctor before they attend their actual consultation with the doctor. Note that the consultation will be held remotely, therefore you will be following the Patient-Centered Interview model for your consultations and you cannot conduct physical examinations on the patient. You are also not allowed to diagnose your patient or prescribe any medicine. Do not entertain the patient if they are acting inappropriate or they ask you to do something outside of this job scope. Remember you are a doctor’s assistant so act like one. The consultation held will only focus on getting information from the patient such as their present illness, past medical history, symptoms and personal information. At the end of the consultation, you will summarize the findings of the consultation in this format \nName: [name]\nGender: [gender]\nPatient Aged: [age]\nMedical History: [medical history].\nSymptoms: [symptoms]\nThis exact format must be followed as the data gathered in the summary will be passed to another program.
# <</SYS>>

# This is the PDF context: {context_str}

# {query_str}
# """)
# text_qa_template = Prompt("""[INST] {context_str} \n\nGiven this above PDF context, please answer my question: {query_str} [/INST] """)
# text_qa_template = Prompt("""<s>[INST] <<SYS>> \nFollowing is the PDF context provided by the user: {context_str}\n<</SYS>> \n\n{query_str} [/INST] """)
# text_qa_template = Prompt("""[INST] {query_str} [/INST] """)

# Query Engine
query_engine = index.as_query_engine(text_qa_template=text_qa_template, streaming=True, service_context=service_context) # with Prompt
# query_engine = index.as_query_engine(streaming=True, service_context=service_context) # without Prompt

## with saving to file

### withoutRAG   oK

In [9]:
# import time

conversation_history = ""
file_path = "/content/drive/MyDrive/RAG/conv_history.txt"

while True:
    user_query = input("User: ")
    if user_query.lower() == "exit":
        with open(file_path, 'a') as file:
            file.write(conversation_history)
        break

    conversation_history += user_query + " [/INST] "

    start_time = time.time()

    response_iter = llm.stream_complete("<s>[INST] " + conversation_history)
    for response in response_iter:
        print(response.delta, end="", flush=True)
        if response.raw['choices'][0]['finish_reason'] == 'stop':
            conversation_history += response.text + " [INST] "

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\nElapsed inference time: {elapsed_time:.2f} s\n")

    # Save conversation to a text file continuously
    # with open(file_path, 'a') as file:
    #     # file.write(conversation_history) was like that
    #     file.write('\n  \n' +user_query+'\n =========== \n'+response.text)
    with open(file_path, 'a') as file:
      # Write the user query
        file.write(f'{user_query}\n')
    # Write the response
        file.write(f'============== \n {response.text}\n\n')

User: What is microbiota?
 Microbiota refers to the trillions of microorganisms that live within a specific environment, such as the human body or a particular ecosystem. The term "microbiota" is derived from the Greek words "micro," meaning small, and "biota," meaning life.
Microbiota are composed of various types of microorganisms, including bacteria, viruses, fungi, and protozoa. These microorganisms are essential for maintaining the proper functioning of various bodily systems and processes, including:
1. Digestion: Microbiota help break down complex nutrients and absorb nutrients from food, such as vitamins and minerals.
2. Immune system function: Microbiota play a crucial role in regulating the immune system and preventing infection. They can also help modulate inflammation and reduce the risk of autoimmune diseases.
3. Barrier function: Microbiota help maintain the integrity of the epithelial barrier, preventing harmful substances from entering the body and protecting against pa


llama_print_timings:        load time =     338.86 ms
llama_print_timings:      sample time =     136.52 ms /   256 runs   (    0.53 ms per token,  1875.24 tokens per second)
llama_print_timings: prompt eval time =     338.73 ms /    16 tokens (   21.17 ms per token,    47.24 tokens per second)
llama_print_timings:        eval time =    5823.01 ms /   255 runs   (   22.84 ms per token,    43.79 tokens per second)
llama_print_timings:       total time =    7377.66 ms /   271 tokens



Elapsed inference time: 7.39 s

User: Explain me step by step which vitaminns are produced


Llama.generate: prefix-match hit


 Microbiota refers to the trillions of microorganisms that reside within and on our bodies, including bacteria, viruses, fungi, and other microorganisms. The human body is home to a diverse ecosystem of microbiota, with different types of microorganisms present in different parts of the body, such as the gut, skin, respiratory tract, and mouth.
Microbiota play a crucial role in maintaining our overall health and well-being. They perform a variety of functions that are essential for our body's proper functioning, including:
1. Digestion: Microbiota in the gut help break down complex nutrients like carbohydrates, proteins, and fats, making them accessible for absorption by the body.
2. Immune system function: Microbiota help regulate the immune system, preventing overactive or underactive immune responses. They also produce immune factors that help protect against infection and disease.
3. Production of vitamins: Microbiota produce certain vitamins that are essential for our body's prope

...


Elapsed inference time: 7.93 s

User: just a question from other field
 Of

Llama.generate: prefix-match hit


 course! Here are some additional questions related to the brain-gut axis:
ղ� Microbiome and mental health?
The brain-gut axis refers to the complex bidirectional communication network between the central nervous system (CNS) and the enteric nervous system (ENS) of the gut. The ENS is responsible for regulating gut function and immune system activity, while the CNS regulates the ENS and influences various physiological processes such as mood, cognitive function, and behavior.
Research has shown that the gut microbiome plays a crucial role in the brain-gut axis and has a significant impact on mental health. An unhealthy gut microbiome has been linked to various psychiatric disorders such as depression, anxiety, and bipolar disorder. On the other hand, a healthy gut microbiome has been associated with improved mood and cognitive function.
Here are some key questions related to the brain-gut axis and the gut microbiome:
1. How does the gut microbiome influence the brain-gut axis and contr


llama_print_timings:        load time =     338.86 ms
llama_print_timings:      sample time =     138.04 ms /   256 runs   (    0.54 ms per token,  1854.51 tokens per second)
llama_print_timings: prompt eval time =     191.54 ms /    11 tokens (   17.41 ms per token,    57.43 tokens per second)
llama_print_timings:        eval time =    6181.83 ms /   255 runs   (   24.24 ms per token,    41.25 tokens per second)
llama_print_timings:       total time =    7917.31 ms /   266 tokens



Elapsed inference time: 7.93 s

User: exit


### without saving to file

In [ ]:
# Inferencing
# Without RAG
conversation_history = ""
while (True):
  user_query = input("User: ")
  if user_query.lower() == "exit":
    break
  conversation_history += user_query + " [/INST] "

  start_time = time.time()

  response_iter = llm.stream_complete("<s>[INST] "+conversation_history)
  for response in response_iter:
    print(response.delta, end="", flush=True)
    # Add to conversation history when response is completed
    if response.raw['choices'][0]['finish_reason'] == 'stop':
      conversation_history += response.text + " [INST] "

  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"\nElapsed inference time: {elapsed_time:.2f} s\n")


In [ ]:
# With RAG
conversation_history = ""
conversation_history += "Hi. [\INST] Hello! You are my scientific assistant. Let's begin the consultation"
# conversation_history += "Hi. [\INST] Hello! I'm the doctor's assistant. Let's begin the consultation, please tell me your name and age."
while (True):
  user_query = input("User: ")
  if user_query.lower() == "exit":
    break
  conversation_history += user_query + " [/INST] "

  start_time = time.time()

  # Query Engine - Default
  response = query_engine.query(conversation_history)
  response.print_response_stream()
  conversation_history += response.response_txt + " [INST] "

  # from pprint import pprint
  # pprint(response)

  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"\nElapsed inference time: {elapsed_time:.2f} s\n")

# Multi-model (Separate pre-screening/conversation and summarization tasks)

In [10]:
# Multi-model (Separate pre-screening/conversation and summarization tasks)
response_iter = llm.stream_complete("""
[INST] <<SYS>>
Summarize the conversation into this format: \nName: [name]\nGender: [gender]\nAge: [age]\nMedical History: [medical history].\nSymptoms: [symptoms]
<</SYS>>
Conversation: """+conversation_history+" [/INST] ")

for response in response_iter:
  print(response.delta, end="", flush=True)

# print(conversation_history)

Llama.generate: prefix-match hit


 Name: Dr. Smith
Gender: Male
Age: 55
Medical History: No significant medical history.
Symptoms: None.
Conversation:
Dr. Smith: Good morning, how can I help you today?
Patient: Hi, Dr. Smith. I'm here to talk to you about my microbiota. Can you tell me what it is?
Dr. Smith: Of course! Microbiota refers to the trillions of microorganisms that live inside and on our bodies. They play a crucial role in our overall health and well-being, influencing everything from our digestion and immune system to our mood and cognitive function.
Patient: That's fascinating! How do microbiota produce vitamins?
Dr. Smith: Microbiota produce certain vitamins, such as vitamin K, which is essential for blood clotting and bone health. They also produce vitamin B12, which is important for nerve function and red blood cell production. Additionally, microbiota can synthesize vitamin D, which is crucial for bone health and immune system function.
Patient


llama_print_timings:        load time =     338.86 ms
llama_print_timings:      sample time =     129.57 ms /   256 runs   (    0.51 ms per token,  1975.71 tokens per second)
llama_print_timings: prompt eval time =     372.70 ms /   226 tokens (    1.65 ms per token,   606.38 tokens per second)
llama_print_timings:        eval time =    6312.78 ms /   255 runs   (   24.76 ms per token,    40.39 tokens per second)
llama_print_timings:       total time =    8161.49 ms /   481 tokens
